#  Using Hierarchical Indexing in Airline On-Time Performance Data

* Pandas supports multidimensional indexes, i.e. indexes which are essentially hierarchical structures.

* This gives us the flexibility to do searches by dimensions.

* As an example we will take [Airline On-Time Performance Data](https://www.transtats.bts.gov/Tables.asp?DB_ID=120&DB_Name=Airline%20On-Time%20Performance%20Data&DB_Short_Name =On-Time#), which are compiled by the [Bureau of Transportation Statistics (BTS)](https://www.bts.gov/) of the US Department of Transportation.

* Contains monthly data of airlines that hold at least one percent of scheduled domestic flight revenue.

* We will take the data for 2019.

In [1]:
import pandas as pd

flights = pd.read_csv('flights.csv.zip', dtype={'DEP_DELAY': float}, parse_dates=['FL_DATE'])

print(flights.shape)
flights.sample(5)

(7422037, 20)


,FL_DATE,TAIL_NUM,CARRIER,ORIGIN,ORIGIN_CITY_NAME,DEST,DEST_CITY_NAME,DEP_TIME,DEP_DELAY,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,DIVERTED,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 19
654562,2019-02-04,N908AE,MQ,ORD,"Chicago, IL",CVG,"Cincinnati, OH",658.0,-7.0,859.0,-20.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3291420,2019-06-14,N900PC,DL,SLC,"Salt Lake City, UT",DCA,"Washington, DC",944.0,-4.0,1555.0,-4.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1568382,2019-03-23,N287AK,AS,PDX,"Portland, OR",EWR,"Newark, NJ",619.0,-1.0,1430.0,-10.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4975119,2019-09-02,N16976,EV,GRK,"Killeen, TX",IAH,"Houston, TX",1545.0,-10.0,1638.0,-32.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
6940890,2019-12-08,N646NK,NK,MCO,"Orlando, FL",BWI,"Baltimore, MD",2247.0,18.0,50.0,13.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN


* Notice that there is a column on the far right called `Unnamed: 19`.

* This is because each line ends with a comma, so `read_csv()` treats it as an empty column.

* We can delete it (with `drop()`).

In [2]:
flights.drop('Unnamed: 19', axis=1, inplace=True)
flights.sample(5)

,FL_DATE,TAIL_NUM,CARRIER,ORIGIN,ORIGIN_CITY_NAME,DEST,DEST_CITY_NAME,DEP_TIME,DEP_DELAY,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,DIVERTED,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
1746202,2019-03-31,N601LR,9E,ATL,"Atlanta, GA",TYS,"Knoxville, TN",2120.0,10.0,2211.0,7.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN
5356909,2019-09-21,N967UW,AA,PHL,"Philadelphia, PA",ATL,"Atlanta, GA",1733.0,23.0,1927.0,8.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN
6619001,2019-11-21,N754EV,OO,DFW,"Dallas/Fort Worth, TX",BTR,"Baton Rouge, LA",1019.0,-6.0,1134.0,-21.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN
3472872,2019-06-23,N7847A,WN,DCA,"Washington, DC",PVD,"Providence, RI",1915.0,5.0,2022.0,-18.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN
2939814,2019-05-29,N376DA,DL,SEA,"Seattle, WA",LAX,"Los Angeles, CA",952.0,-3.0,1234.0,-6.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN


* Now let's build a hierarchical index using the following fields:
 * `CARRIER`
 * `ORIGIN`
 * `DEST`
 * `TAIL_NUM`
 * `FL_DATE`

In [3]:
hierarchical_flights = flights.set_index(['CARRIER', 'ORIGIN', 
                                          'DEST', 'TAIL_NUM', 
                                          'FL_DATE'])

hierarchical_flights = hierarchical_flights.sort_index(level=['CARRIER', 'ORIGIN', 
                                                              'DEST', 'TAIL_NUM', 
                                                              'FL_DATE'])

hierarchical_flights

ORIGIN_CITY_NAME  \
CARRIER ORIGIN DEST TAIL_NUM FL_DATE                                      
9E      ABE    ATL  N131EV   2019-02-22  Allentown/Bethlehem/Easton, PA   
                             2019-04-19  Allentown/Bethlehem/Easton, PA   
                             2019-07-10  Allentown/Bethlehem/Easton, PA   
                    N132EV   2019-02-28  Allentown/Bethlehem/Easton, PA   
                             2019-04-09  Allentown/Bethlehem/Easton, PA   
...                                                                 ...   
YX      XNA    PHL  N440YX   2019-11-09                Fayetteville, AR   
                    N442YX   2019-09-04                Fayetteville, AR   
                             2019-11-20                Fayetteville, AR   
                    N445YX   2019-09-16                Fayetteville, AR   
                             2019-10-07                Fayetteville, AR   

                                           DEST_CITY_NAME  DEP_TIME  \
CARRIER ORIGIN DEST TAIL_NUM FL_DATE                                  
9E      ABE    ATL  N131EV   2019-02-22       Atlanta, GA    1319.0   
                             2019-04-19       Atlanta, GA    1308.0   
                             2019-07-10       Atlanta, GA    1754.0   
                    N132EV   2019-02-28       Atlanta, GA    1745.0   
                             2019-04-09       Atlanta, GA    1841.0   
...                                                   ...       ...   
YX      XNA    PHL  N440YX   2019-11-09  Philadelphia, PA    1552.0   
                    N442YX   2019-09-04  Philadelphia, PA    1553.0   
                             2019-11-20  Philadelphia, PA    1551.0   
                    N445YX   2019-09-16  Philadelphia, PA    1547.0   
                             2019-10-07  Philadelphia, PA    1545.0   

                                         DEP_DELAY  ARR_TIME  ARR_DELAY  \
CARRIER ORIGIN DEST TAIL_NUM FL_DATE                                      
9E      ABE    ATL  N131EV   2019-02-22        9.0    1545.0       21.0   
                             2019-04-19        2.0    1546.0       25.0   
                             2019-07-10       -1.0    1951.0      -15.0   
                    N132EV   2019-02-28       -8.0    1957.0      -17.0   
                             2019-04-09       43.0    2148.0       94.0   
...                                            ...       ...        ...   
YX      XNA    PHL  N440YX   2019-11-09       -6.0    1956.0       11.0   
                    N442YX   2019-09-04       -2.0    2016.0       27.0   
                             2019-11-20       -7.0    1926.0      -19.0   
                    N445YX   2019-09-16       -7.0    1935.0      -13.0   
                             2019-10-07       -9.0    1930.0      -17.0   

                                         CANCELLED CANCELLATION_CODE  \
CARRIER ORIGIN DEST TAIL_NUM FL_DATE                                   
9E      ABE    ATL  N131EV   2019-02-22        0.0               NaN   
                             2019-04-19        0.0               NaN   
                             2019-07-10        0.0               NaN   
                    N132EV   2019-02-28        0.0               NaN   
                             2019-04-09        0.0               NaN   
...                                            ...               ...   
YX      XNA    PHL  N440YX   2019-11-09        0.0               NaN   
                    N442YX   2019-09-04        0.0               NaN   
                             2019-11-20        0.0               NaN   
                    N445YX   2019-09-16        0.0               NaN   
                             2019-10-07        0.0               NaN   

                                         DIVERTED  CARRIER_DELAY  \
CARRIER ORIGIN DEST TAIL_NUM FL_DATE                               
9E      ABE    ATL  N131EV   2019-02-22       0.0            9.0   
                             2019-04-1

* A hierarchical index is an object of type `MultiIndex`.

* The levels of a `MultiIndex` are the names of the columns we used to build it.

* We can refer to these either by the names themselves or by their position (starting from 0 for the first from the left).

* So if we want to count the flights by company we will give:

In [4]:
hierarchical_flights.index.get_level_values('CARRIER').value_counts()

WN    1363946
DL     991986
AA     946776
OO     836445
UA     625910
YX     329149
MQ     327007
B6     297411
OH     289304
AS     264816
9E     257132
YV     227888
NK     204845
F9     135543
EV     134683
G4     105305
HA      83891
Name: CARRIER, dtype: int64

* Or equivalently,

In [5]:
hierarchical_flights.index.get_level_values(0).value_counts()

WN    1363946
DL     991986
AA     946776
OO     836445
UA     625910
YX     329149
MQ     327007
B6     297411
OH     289304
AS     264816
9E     257132
YV     227888
NK     204845
F9     135543
EV     134683
G4     105305
HA      83891
Name: CARRIER, dtype: int64

* Let's find the busiest airports in terms of departures:

In [6]:
hierarchical_flights.index.get_level_values('ORIGIN').value_counts()[:10] 

ATL    395009
ORD    339606
DFW    304344
DEN    252026
CLT    235496
LAX    219952
IAH    179688
PHX    175328
LGA    171665
SFO    170918
Name: ORIGIN, dtype: int64

* Regarding arrivals:

In [7]:
hierarchical_flights.index.get_level_values('DEST').value_counts()[:10] 

ATL    395026
ORD    339569
DFW    304346
DEN    252064
CLT    235490
LAX    219996
IAH    179682
PHX    175343
LGA    171665
SFO    170966
Name: DEST, dtype: int64

* The `DEP_DELAY` field has both positive and negative values. We want to see what happens to the positive ones.

In [8]:
delayed_flights = hierarchical_flights.loc[hierarchical_flights['DEP_DELAY'] > 0]
delayed_flights['DEP_DELAY']

CARRIER  ORIGIN  DEST  TAIL_NUM  FL_DATE   
9E       ABE     ATL   N131EV    2019-02-22      9.0
                                 2019-04-19      2.0
                       N132EV    2019-04-09     43.0
                       N133EV    2019-04-25     10.0
                                 2019-12-23     49.0
                                               ...  
YX       XNA     PHL   N138HQ    2019-09-06    107.0
                                 2019-12-13     36.0
                       N416YX    2019-09-28      3.0
                       N428YX    2019-10-16     22.0
                       N433YX    2019-12-01    116.0
Name: DEP_DELAY, Length: 2521492, dtype: float64

* From the above we can make a "misery ranking", i.e. the airports which seem to be champions in delays.

In [9]:
delayed_flights_by_origin = delayed_flights.groupby(level='ORIGIN')['DEP_DELAY']\
    .agg(['median', 'mean', 'count'])\
    .sort_values(by='median', ascending=False)
delayed_flights_by_origin[:10]

,median,mean,count
ORIGIN,,,
DVL,77.0,141.301587,126
SLN,59.0,110.613546,251
SHD,57.0,116.976608,171
HYS,51.0,117.920266,301
VEL,50.0,88.461538,130
UIN,48.0,90.852071,169
ATY,48.0,113.400000,105
OTH,47.0,89.480519,154
ACV,46.0,91.726210,599


* It would be good to see the percentage of flights that are delayed.

* So let's take the total flights.

In [10]:
airport_traffic = hierarchical_flights.groupby(level='ORIGIN').size().to_frame(name='total_departures')
airport_traffic.sort_values(by='total_departures', ascending=False)[:10]

,total_departures
ORIGIN,
ATL,395009
ORD,339606
DFW,304344
DEN,252026
CLT,235496
LAX,219952
IAH,179688
PHX,175328
LGA,171665


* By joining the two DataFrames we can have all the information in one.

In [11]:
delayed_flights_by_origin = pd.merge(delayed_flights_by_origin,
                                     airport_traffic, left_index=True, right_index=True)
delayed_flights_by_origin

,median,mean,count,total_departures
ORIGIN,,,,
DVL,77.0,141.301587,126,625
SLN,59.0,110.613546,251,938
SHD,57.0,116.976608,171,708
HYS,51.0,117.920266,301,1251
VEL,50.0,88.461538,130,626
...,...,...,...,...
LIH,9.0,30.815107,3548,14063
OGG,9.0,27.812275,8342,27326
AKN,8.0,11.916667,12,61


* And now we can calculate the percentage of delayed flights (in terms of departure).

In [12]:
delayed_flights_by_origin['pct_delayed'] = \
    100 * delayed_flights_by_origin['count'] / delayed_flights_by_origin['total_departures']
delayed_flights_by_origin

,median,mean,count,total_departures,pct_delayed
ORIGIN,,,,,
DVL,77.0,141.301587,126,625,20.160000
SLN,59.0,110.613546,251,938,26.759062
SHD,57.0,116.976608,171,708,24.152542
HYS,51.0,117.920266,301,1251,24.060751
VEL,50.0,88.461538,130,626,20.766773
...,...,...,...,...,...
LIH,9.0,30.815107,3548,14063,25.229325
OGG,9.0,27.812275,8342,27326,30.527703
AKN,8.0,11.916667,12,61,19.672131


* With a hierarchical index we can filter our data easily, without having to use logical comparison conditions.

* If we want to find the flights made by Southwest Airlines (`WN`), Delta (`DL`), and American Airlines (`AA`) it is enough to give:

In [13]:
hierarchical_flights.loc[['WN', 'DL', 'AA'], ['DEP_TIME', 'DEP_DELAY']]

DEP_TIME  DEP_DELAY
CARRIER ORIGIN DEST TAIL_NUM FL_DATE                        
WN      ABQ    AUS  N204WN   2019-09-01    1204.0       -6.0
                    N205WN   2019-08-03    1519.0        4.0
                    N211WN   2019-02-10    1950.0        5.0
                    N212WN   2019-03-03     811.0       71.0
                    N219WN   2019-07-21    1920.0       20.0
...                                           ...        ...
AA      XNA    ORD  N871NN   2019-06-07    1541.0      -10.0
                    N878NN   2019-06-07    1737.0       -6.0
                    N988NN   2019-06-04    1606.0       15.0
               PHL  N824AW   2019-06-03     710.0       80.0
                    N832AW   2019-06-08    1516.0      -14.0

[3302708 rows x 2 columns]

* Thanks to the hierarchical index we can make easy and more complex queries.

* we will select flights operated by Southwest Airlines (`WN`), Delta (`DL`), and American Airlines (`AA`) departing from New York JFK (`JFK`) or New York La Guardia (`LGA `).

In [14]:
hierarchical_flights.loc[(['WN', 'DL', 'AA'], ['JFK', 'LGA']), 
                         ['DEP_TIME', 'DEP_DELAY']]

DEP_TIME  DEP_DELAY
CARRIER ORIGIN DEST TAIL_NUM FL_DATE                        
WN      LGA    ATL  NaN      2019-01-20       NaN        NaN
                             2019-02-12       NaN        NaN
                             2019-02-21       NaN        NaN
                             2019-03-04       NaN        NaN
                    N200WN   2019-02-27     556.0       -4.0
...                                           ...        ...
AA      LGA    RDU  N967UW   2019-12-09    1738.0       28.0
                             2019-12-15    1952.0       -8.0
                             2019-12-18    1820.0       10.0
                             2019-12-19     956.0       -3.0
                             2019-12-24    1159.0       -6.0

[113095 rows x 2 columns]

* Let's say we are interested in the subset of the above arriving at San Francisco (`SFO'):

In [15]:
hierarchical_flights.loc[(['WN', 'DL', 'AA'], ['JFK', 'LGA'], 'SFO'), 
                         ['DEP_TIME', 'DEP_DELAY']]

DEP_TIME  DEP_DELAY
CARRIER ORIGIN DEST TAIL_NUM FL_DATE                        
DL      JFK    SFO  N152DL   2019-03-10     718.0       18.0
                             2019-04-12    1723.0        8.0
                    N154DL   2019-08-16    2240.0       90.0
                    N155DL   2019-04-26     957.0       17.0
                    N156DL   2019-07-29     810.0       -5.0
...                                           ...        ...
AA      JFK    SFO  N117AN   2019-12-14     652.0       -8.0
                             2019-12-16    1058.0       -2.0
                             2019-12-21    1718.0        9.0
                             2019-12-31    1706.0       -3.0
                    N828AW   2019-10-13    1635.0       15.0

[4096 rows x 2 columns]

* And if we want to find all flights between these airports, regardless of company?

* We can do this easily using an `IndexSlice`.

In [16]:
hierarchical_flights.loc[pd.IndexSlice[:, ['JFK', 'LGA'], 'SFO'], 
                         ['DEP_TIME', 'DEP_DELAY']]

DEP_TIME  DEP_DELAY
CARRIER ORIGIN DEST TAIL_NUM FL_DATE                        
AA      JFK    SFO  N101NN   2019-01-06    1252.0      112.0
                             2019-01-13    1833.0        3.0
                             2019-01-18    1823.0       -7.0
                             2019-01-22    1857.0       27.0
                             2019-01-25    1823.0       -7.0
...                                           ...        ...
DL      JFK    SFO  N844MH   2019-02-15     924.0       -6.0
                    N845MH   2019-01-09    1100.0       90.0
                             2019-02-13     924.0       -6.0
                             2019-02-24     925.0       -5.0
                             2019-08-17     803.0       -7.0

[7620 rows x 2 columns]

* Or we might want to find the subset of the above flights that took place in the first fortnight of January.

* Remember that the last level of the index was the date of the flight.

In [17]:
hierarchical_flights.loc[pd.IndexSlice[:, ['JFK', 'LGA'], 'SFO', :, 
                                       '2019-01-01':'2019-01-15'],
                         ['DEP_TIME', 'DEP_DELAY', 'ARR_TIME', 'ARR_DELAY']]

DEP_TIME  DEP_DELAY  ARR_TIME  \
CARRIER ORIGIN DEST TAIL_NUM FL_DATE                                     
AA      JFK    SFO  N101NN   2019-01-06    1252.0      112.0    1702.0   
                             2019-01-13    1833.0        3.0    2128.0   
                    N102NN   2019-01-08    1057.0       -3.0    1507.0   
                             2019-01-10    1053.0       -7.0    1423.0   
                    N103NN   2019-01-10     656.0       -4.0    1031.0   
...                                           ...        ...       ...   
DL      JFK    SFO  N842MH   2019-01-09    1551.0       -4.0    1921.0   
                             2019-01-10    1939.0       14.0    2336.0   
                    N843MH   2019-01-13     923.0       -7.0    1238.0   
                             2019-01-15    1158.0      148.0    1529.0   
                    N845MH   2019-01-09    1100.0       90.0    1430.0   

                                         ARR_DELAY  
CARRIER ORIGIN DEST TAIL_NUM FL_DATE                
AA      JFK    SFO  N101NN   2019-01-06      141.0  
                             2019-01-13      -49.0  
                    N102NN   2019-01-08       26.0  
                             2019-01-10      -18.0  
                    N103NN   2019-01-10       -9.0  
...                                            ...  
DL      JFK    SFO  N842MH   2019-01-09      -42.0  
                             2019-01-10       11.0  
                    N843MH   2019-01-13      -49.0  
                             2019-01-15      122.0  
                    N845MH   2019-01-09       63.0  

[301 rows x 4 columns]